In [7]:
from openai import OpenAI

def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content.rstrip('\n')


def openai_chat(messages):
    client = OpenAI(api_key=read_file("/home/teveritt/OpenAI-API-DCU-AI.key"))

    return client.chat.completions.create(
        model='gpt-4o',
        temperature=0.0,
        messages=messages,
        stream=False,
        max_tokens=1525
    )

In [34]:
def fix_text(text):
    m = [
        {
            "role": "system",
            "content": "This GPT is a job description fixer that restructures provided text without altering the words, grammar, or their meanings. It preserves British and American spellings.  The focus is only to fix the spacing, separators, and list items. Nothing else.  It removes characters that are noise and not part of the context."
        }, {
            "role": "user",
            "content": text
        }
    ]
    output = openai_chat(m)
    content = output.choices[0].message.content.strip()
    content = content.replace('***', '')
    content = content.replace('**', '')
    content = content.replace('*', '-')
    return content

In [51]:
import os

with open('processed-fix-text.txt', 'r') as file:
    processed = set()
    for line in file.readlines():
        processed.add(line.strip().replace('\n',''))

processed = list(processed)
processed.sort()
print(f'Processed: {processed}')

for type in ['train','val', 'test']:
    dataframes = []
    for file_name in os.listdir(type):
        file_name, file_extension = os.path.splitext(file_name)
        base_path = os.path.join(type, file_name)
        if f'{base_path}.txt' in processed:
            print(f'Already processed {base_path}.txt')
        else:
            if file_name.startswith('Kaggle'):
                if os.path.isfile(f'{base_path}.txt'):
                        print(f'Processing {base_path}.txt')
                        with open(f'{base_path}.txt', 'r') as file:
                            text = file.read().strip()
                            text = fix_text(text)
                        with open(f'{base_path}.txt', 'w') as f:
                            f.write(text)
                        
                        processed.append(f'{base_path}.txt')
                        with open('processed-fix-text.txt', 'w') as file:
                            for item in processed:
                                file.write('%s\n' % item)
                            
        #break
    #break
                
